In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_raw = pd.read_excel("/content/drive/MyDrive/data/split_purchase_data.xlsx")

In [ ]:
df_raw.head(10)

,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item,Item Category,Vendor,Total Value
0,Dept001,PR_10001001,2022-01-12,Equipment AA,Equipment,V0001,2000.0
1,Dept001,PR_10001002,2022-01-15,Equipment AA,Equipment,V0001,4000.0
2,Dept001,PR_10001003,2022-01-14,Equipment AB,Equipment,V0002,3000.0
3,Dept001,PR_10001004,2022-02-14,Equipment AB,Equipment,V0003,6000.0
4,Dept001,PR_10001005,2022-02-15,Equipment AB,Equipment,V0004,8000.0
5,Dept002,PR_10001006,2022-01-16,Equipment AA,Equipment,V0001,4000.0
6,Dept002,PR_10001007,2022-01-17,Equipment AB,Equipment,V0002,3000.0


In [ ]:
df_raw['Purchase Requisition Date'] = pd.to_datetime(df_raw['Purchase Requisition Date'])
df_raw['Purchase Requisition Number'] = df_raw['Purchase Requisition Number'].astype(str)

In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Purchasing Department        7 non-null      object        
 1   Purchase Requisition Number  7 non-null      object        
 2   Purchase Requisition Date    7 non-null      datetime64[ns]
 3   Item                         7 non-null      object        
 4   Item Category                7 non-null      object        
 5   Vendor                       7 non-null      object        
 6   Total Value                  7 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 520.0+ bytes


In [ ]:
item_field = 'Item'

In [ ]:
df_dept_item_sorted = df_raw.sort_values(['Purchasing Department', item_field,'Purchase Requisition Date'])

In [ ]:
df_dept_item_sorted['Prev PR Date'] = df_dept_item_sorted['Purchase Requisition Date'].shift()

In [ ]:
df_dept_item_sorted['PreGroup'] = df_dept_item_sorted[['Purchasing Department', item_field,'Purchase Requisition Date']]\
.groupby(['Purchasing Department', item_field]).cumcount()

In [ ]:
df_dept_item_sorted.head()

,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item,Item Category,Vendor,Total Value,Prev PR Date,PreGroup
0,Dept001,PR_10001001,2022-01-12,Equipment AA,Equipment,V0001,2000.0,NaT,0
1,Dept001,PR_10001002,2022-01-15,Equipment AA,Equipment,V0001,4000.0,2022-01-12,1
2,Dept001,PR_10001003,2022-01-14,Equipment AB,Equipment,V0002,3000.0,2022-01-15,0
3,Dept001,PR_10001004,2022-02-14,Equipment AB,Equipment,V0003,6000.0,2022-01-14,1
4,Dept001,PR_10001005,2022-02-15,Equipment AB,Equipment,V0004,8000.0,2022-02-14,2


In [ ]:
mask = df_dept_item_sorted['PreGroup'] == 0
df_dept_item_sorted.loc[mask,'Prev PR Date'] = pd.NaT

In [ ]:
#Calculating number of days difference between adjacent records
df_dept_item_sorted['Date_Diff'] = (df_dept_item_sorted['Purchase Requisition Date'] - df_dept_item_sorted['Prev PR Date']).dt.days

In [ ]:
#Set Threshold 
n = 5
df_dept_item_sorted['Flag Out of Range'] = (1 - (df_dept_item_sorted['Date_Diff'] <= n)).astype(int)

In [ ]:
df_dept_item_sorted['Grouping'] = df_dept_item_sorted['Flag Out of Range'].cumsum()

In [ ]:
df_dept_item_sorted.head()

,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item,Item Category,Vendor,Total Value,Prev PR Date,PreGroup,Date_Diff,Flag Out of Range,Grouping
0,Dept001,PR_10001001,2022-01-12,Equipment AA,Equipment,V0001,2000.0,NaT,0,NaN,1,1
1,Dept001,PR_10001002,2022-01-15,Equipment AA,Equipment,V0001,4000.0,2022-01-12,1,3.0,0,1
2,Dept001,PR_10001003,2022-01-14,Equipment AB,Equipment,V0002,3000.0,NaT,0,NaN,1,2
3,Dept001,PR_10001004,2022-02-14,Equipment AB,Equipment,V0003,6000.0,2022-01-14,1,31.0,1,3
4,Dept001,PR_10001005,2022-02-15,Equipment AB,Equipment,V0004,8000.0,2022-02-14,2,1.0,0,3


In [ ]:
df_pr_count = df_dept_item_sorted[['Grouping',item_field,'Purchase Requisition Number']].groupby(['Grouping',item_field]).nunique().reset_index()

In [ ]:
df_pr_count.columns = ['Grouping', item_field, 'Count PR']

In [ ]:
# Identifies the groups that are valid split purchase cases
df_split_purchase_group = df_pr_count[df_pr_count['Count PR']>=2]

In [ ]:
# Joining of Split Purchase Groups to initial details
df_split_purchase_details = df_split_purchase_group.merge(df_dept_item_sorted,
                                                          how='inner',
                                                          on= ['Grouping',item_field])

In [ ]:
df_split_purchase_details.head()

,Grouping,Item,Count PR,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item Category,Vendor,Total Value,Prev PR Date,PreGroup,Date_Diff,Flag Out of Range
0,1,Equipment AA,2,Dept001,PR_10001001,2022-01-12,Equipment,V0001,2000.0,NaT,0,NaN,1
1,1,Equipment AA,2,Dept001,PR_10001002,2022-01-15,Equipment,V0001,4000.0,2022-01-12,1,3.0,0
2,3,Equipment AB,2,Dept001,PR_10001004,2022-02-14,Equipment,V0003,6000.0,2022-01-14,1,31.0,1
3,3,Equipment AB,2,Dept001,PR_10001005,2022-02-15,Equipment,V0004,8000.0,2022-02-14,2,1.0,0


In [ ]:
df_split_purchase_details['Date_Diff'] = np.where(df_split_purchase_details['Flag Out of Range'] == 1,
                                                  pd.NaT,
                                                  df_split_purchase_details['Date_Diff'])

In [ ]:
df_split_purchase_details.sort_values(['Grouping','Purchase Requisition Number'], inplace=True)

In [ ]:
df_split_purchase_desc = df_split_purchase_details.groupby(['Grouping',item_field])\
.agg({'Purchase Requisition Number': '|'.join}).reset_index()

In [ ]:
df_split_purchase_desc.columns = ['Grouping', item_field, 'PR with similar items']

In [ ]:
df_split_purchase_comb = df_split_purchase_details.merge(df_split_purchase_desc,
                                                         how='inner',
                                                         on=['Grouping',item_field])

In [ ]:
df_split_purchase_comb

,Grouping,Item,Count PR,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item Category,Vendor,Total Value,Prev PR Date,PreGroup,Date_Diff,Flag Out of Range,PR with similar items
0,1,Equipment AA,2,Dept001,PR_10001001,2022-01-12,Equipment,V0001,2000.0,NaT,0,NaT,1,PR_10001001|PR_10001002
1,1,Equipment AA,2,Dept001,PR_10001002,2022-01-15,Equipment,V0001,4000.0,2022-01-12,1,3.0,0,PR_10001001|PR_10001002
2,3,Equipment AB,2,Dept001,PR_10001004,2022-02-14,Equipment,V0003,6000.0,2022-01-14,1,NaT,1,PR_10001004|PR_10001005
3,3,Equipment AB,2,Dept001,PR_10001005,2022-02-15,Equipment,V0004,8000.0,2022-02-14,2,1.0,0,PR_10001004|PR_10001005


### Derive futher prioritization based on rules

- Was the threshold avoided due to the split ?
- Are the splits belonging to the same vendor ?
- Are the dates diff within one day ?

In [ ]:
df_priority_score = df_split_purchase_comb.groupby(['Grouping']).agg({
    'Total Value':['min','sum'],
    'Date_Diff':'min',
    'Vendor':['nunique','count']
}).reset_index()



In [ ]:
df_priority_score.columns = ['Grouping', 'Min_Value', 'Max_Value', 'Min_Date_Diff', 'Unique_Vendor', 'Count_Vendor']

In [ ]:
value_threshold = 5_000
df_priority_score['Flag_Threshold_Crossed'] = \
((df_priority_score['Min_Value'] < value_threshold) & \
(df_priority_score['Max_Value'] >= value_threshold)).astype(int)

In [ ]:
df_priority_score['Flag_Same_Vendor'] = \
((df_priority_score['Unique_Vendor'] == df_priority_score['Count_Vendor'])).astype(int)

In [ ]:
days_threshold = 1
df_priority_score['Flag_Min_Days'] = \
(df_priority_score['Min_Date_Diff'] <= days_threshold).astype(int)

In [ ]:
df_priority_score['Score'] = \
df_priority_score['Flag_Threshold_Crossed'] + \
df_priority_score['Flag_Same_Vendor'] + \
df_priority_score['Flag_Min_Days']


In [ ]:
df_priority_score

,Grouping,Min_Value,Max_Value,Min_Date_Diff,Unique_Vendor,Count_Vendor,Flag_Threshold_Crossed,Flag_Same_Vendor,Flag_Min_Days,Score
0,1,2000.0,6000.0,3.0,1,2,1,0,0,1
1,3,6000.0,14000.0,1.0,2,2,0,1,1,2


In [ ]:
df_sop_item = df_split_purchase_comb.merge(df_priority_score,
                                           how='left',
                                           on='Grouping')

In [ ]:
df_sop_item.insert(0,'Item Type',f"1_{item_field}")

In [ ]:
df_sop_item.head()

,Item Type,Grouping,Item,Count PR,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item Category,Vendor,Total Value,...,PR with similar items,Min_Value,Max_Value,Min_Date_Diff,Unique_Vendor,Count_Vendor,Flag_Threshold_Crossed,Flag_Same_Vendor,Flag_Min_Days,Score
0,1_Item,1,Equipment AA,2,Dept001,PR_10001001,2022-01-12,Equipment,V0001,2000.0,...,PR_10001001|PR_10001002,2000.0,6000.0,3.0,1,2,1,0,0,1
1,1_Item,1,Equipment AA,2,Dept001,PR_10001002,2022-01-15,Equipment,V0001,4000.0,...,PR_10001001|PR_10001002,2000.0,6000.0,3.0,1,2,1,0,0,1
2,1_Item,3,Equipment AB,2,Dept001,PR_10001004,2022-02-14,Equipment,V0003,6000.0,...,PR_10001004|PR_10001005,6000.0,14000.0,1.0,2,2,0,1,1,2
3,1_Item,3,Equipment AB,2,Dept001,PR_10001005,2022-02-15,Equipment,V0004,8000.0,...,PR_10001004|PR_10001005,6000.0,14000.0,1.0,2,2,0,1,1,2


## Performing the calculations for Split Purchase based on Item Categories

In [ ]:
item_field = 'Item Category'

In [ ]:
df_dept_item_sorted = df_raw.sort_values(['Purchasing Department', item_field,'Purchase Requisition Date'])

In [ ]:
df_dept_item_sorted['Prev PR Date'] = df_dept_item_sorted['Purchase Requisition Date'].shift()

In [ ]:
df_dept_item_sorted['PreGroup'] = df_dept_item_sorted[['Purchasing Department', item_field,'Purchase Requisition Date']]\
.groupby(['Purchasing Department', item_field]).cumcount()

In [ ]:
df_dept_item_sorted.head()

,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item,Item Category,Vendor,Total Value,Prev PR Date,PreGroup
0,Dept001,PR_10001001,2022-01-12,Equipment AA,Equipment,V0001,2000.0,NaT,0
2,Dept001,PR_10001003,2022-01-14,Equipment AB,Equipment,V0002,3000.0,2022-01-12,1
1,Dept001,PR_10001002,2022-01-15,Equipment AA,Equipment,V0001,4000.0,2022-01-14,2
3,Dept001,PR_10001004,2022-02-14,Equipment AB,Equipment,V0003,6000.0,2022-01-15,3
4,Dept001,PR_10001005,2022-02-15,Equipment AB,Equipment,V0004,8000.0,2022-02-14,4


In [ ]:
mask = df_dept_item_sorted['PreGroup'] == 0
df_dept_item_sorted.loc[mask,'Prev PR Date'] = pd.NaT

In [ ]:
#Calculating number of days difference between adjacent records
df_dept_item_sorted['Date_Diff'] = (df_dept_item_sorted['Purchase Requisition Date'] - df_dept_item_sorted['Prev PR Date']).dt.days

In [ ]:
#Set Threshold 
n = 5
df_dept_item_sorted['Flag Out of Range'] = (1 - (df_dept_item_sorted['Date_Diff'] <= n)).astype(int)

In [ ]:
df_dept_item_sorted['Grouping'] = df_dept_item_sorted['Flag Out of Range'].cumsum()

In [ ]:
df_dept_item_sorted.head()

,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item,Item Category,Vendor,Total Value,Prev PR Date,PreGroup,Date_Diff,Flag Out of Range,Grouping
0,Dept001,PR_10001001,2022-01-12,Equipment AA,Equipment,V0001,2000.0,NaT,0,NaN,1,1
2,Dept001,PR_10001003,2022-01-14,Equipment AB,Equipment,V0002,3000.0,2022-01-12,1,2.0,0,1
1,Dept001,PR_10001002,2022-01-15,Equipment AA,Equipment,V0001,4000.0,2022-01-14,2,1.0,0,1
3,Dept001,PR_10001004,2022-02-14,Equipment AB,Equipment,V0003,6000.0,2022-01-15,3,30.0,1,2
4,Dept001,PR_10001005,2022-02-15,Equipment AB,Equipment,V0004,8000.0,2022-02-14,4,1.0,0,2


In [ ]:
df_pr_count = df_dept_item_sorted[['Grouping',item_field,'Purchase Requisition Number']].groupby(['Grouping',item_field]).nunique().reset_index()

In [ ]:
df_pr_count.columns = ['Grouping', item_field, 'Count PR']

In [ ]:
# Identifies the groups that are valid split purchase cases
df_split_purchase_group = df_pr_count[df_pr_count['Count PR']>=2]

In [ ]:
# Joining of Split Purchase Groups to initial details
df_split_purchase_details = df_split_purchase_group.merge(df_dept_item_sorted,
                                                          how='inner',
                                                          on= ['Grouping',item_field])

In [ ]:
df_split_purchase_details.head()

,Grouping,Item Category,Count PR,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item,Vendor,Total Value,Prev PR Date,PreGroup,Date_Diff,Flag Out of Range
0,1,Equipment,3,Dept001,PR_10001001,2022-01-12,Equipment AA,V0001,2000.0,NaT,0,NaN,1
1,1,Equipment,3,Dept001,PR_10001003,2022-01-14,Equipment AB,V0002,3000.0,2022-01-12,1,2.0,0
2,1,Equipment,3,Dept001,PR_10001002,2022-01-15,Equipment AA,V0001,4000.0,2022-01-14,2,1.0,0
3,2,Equipment,2,Dept001,PR_10001004,2022-02-14,Equipment AB,V0003,6000.0,2022-01-15,3,30.0,1
4,2,Equipment,2,Dept001,PR_10001005,2022-02-15,Equipment AB,V0004,8000.0,2022-02-14,4,1.0,0


In [ ]:
df_split_purchase_details['Date_Diff'] = np.where(df_split_purchase_details['Flag Out of Range'] == 1,
                                                  pd.NaT,
                                                  df_split_purchase_details['Date_Diff'])

In [ ]:
df_split_purchase_details.sort_values(['Grouping','Purchase Requisition Number'], inplace=True)

In [ ]:
df_split_purchase_desc = df_split_purchase_details.groupby(['Grouping',item_field])\
.agg({'Purchase Requisition Number': '|'.join}).reset_index()

In [ ]:
df_split_purchase_desc.columns = ['Grouping', item_field, 'PR with similar items']

In [ ]:
df_split_purchase_comb = df_split_purchase_details.merge(df_split_purchase_desc,
                                                         how='inner',
                                                         on=['Grouping',item_field])

In [ ]:
df_split_purchase_comb

,Grouping,Item Category,Count PR,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item,Vendor,Total Value,Prev PR Date,PreGroup,Date_Diff,Flag Out of Range,PR with similar items
0,1,Equipment,3,Dept001,PR_10001001,2022-01-12,Equipment AA,V0001,2000.0,NaT,0,NaT,1,PR_10001001|PR_10001002|PR_10001003
1,1,Equipment,3,Dept001,PR_10001002,2022-01-15,Equipment AA,V0001,4000.0,2022-01-14,2,1.0,0,PR_10001001|PR_10001002|PR_10001003
2,1,Equipment,3,Dept001,PR_10001003,2022-01-14,Equipment AB,V0002,3000.0,2022-01-12,1,2.0,0,PR_10001001|PR_10001002|PR_10001003
3,2,Equipment,2,Dept001,PR_10001004,2022-02-14,Equipment AB,V0003,6000.0,2022-01-15,3,NaT,1,PR_10001004|PR_10001005
4,2,Equipment,2,Dept001,PR_10001005,2022-02-15,Equipment AB,V0004,8000.0,2022-02-14,4,1.0,0,PR_10001004|PR_10001005
5,3,Equipment,2,Dept002,PR_10001006,2022-01-16,Equipment AA,V0001,4000.0,NaT,0,NaT,1,PR_10001006|PR_10001007
6,3,Equipment,2,Dept002,PR_10001007,2022-01-17,Equipment AB,V0002,3000.0,2022-01-16,1,1.0,0,PR_10001006|PR_10001007


### Derive futher prioritization based on rules

- Was the threshold avoided due to the split ?
- Are the splits belonging to the same vendor ?
- Are the dates diff within one day ?

In [ ]:
df_priority_score = df_split_purchase_comb.groupby(['Grouping']).agg({
    'Total Value':['min','sum'],
    'Date_Diff':'min',
    'Vendor':['nunique','count']
}).reset_index()

In [ ]:
df_priority_score.columns = ['Grouping', 'Min_Value', 'Max_Value', 'Min_Date_Diff', 'Unique_Vendor', 'Count_Vendor']

In [ ]:
value_threshold = 5_000
df_priority_score['Flag_Threshold_Crossed'] = \
((df_priority_score['Min_Value'] < value_threshold) & \
(df_priority_score['Max_Value'] >= value_threshold)).astype(int)

In [ ]:
df_priority_score['Flag_Same_Vendor'] = \
((df_priority_score['Unique_Vendor'] == df_priority_score['Count_Vendor'])).astype(int)

In [ ]:
days_threshold = 1
df_priority_score['Flag_Min_Days'] = \
(df_priority_score['Min_Date_Diff'] <= days_threshold).astype(int)

In [ ]:
df_priority_score['Score'] = \
df_priority_score['Flag_Threshold_Crossed'] + \
df_priority_score['Flag_Same_Vendor'] + \
df_priority_score['Flag_Min_Days']


In [ ]:
df_priority_score

,Grouping,Min_Value,Max_Value,Min_Date_Diff,Unique_Vendor,Count_Vendor,Flag_Threshold_Crossed,Flag_Same_Vendor,Flag_Min_Days,Score
0,1,2000.0,9000.0,1.0,2,3,1,0,1,2
1,2,6000.0,14000.0,1.0,2,2,0,1,1,2
2,3,3000.0,7000.0,1.0,2,2,1,1,1,3


In [ ]:
df_sop_itemcat = df_split_purchase_comb.merge(df_priority_score,
                                           how='left',
                                           on='Grouping')

In [ ]:
df_sop_itemcat.insert(0,'Item Type',f"2_{item_field}")

In [ ]:
df_sop_itemcat.head()

,Item Type,Grouping,Item Category,Count PR,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item,Vendor,Total Value,...,PR with similar items,Min_Value,Max_Value,Min_Date_Diff,Unique_Vendor,Count_Vendor,Flag_Threshold_Crossed,Flag_Same_Vendor,Flag_Min_Days,Score
0,2_Item Category,1,Equipment,3,Dept001,PR_10001001,2022-01-12,Equipment AA,V0001,2000.0,...,PR_10001001|PR_10001002|PR_10001003,2000.0,9000.0,1.0,2,3,1,0,1,2
1,2_Item Category,1,Equipment,3,Dept001,PR_10001002,2022-01-15,Equipment AA,V0001,4000.0,...,PR_10001001|PR_10001002|PR_10001003,2000.0,9000.0,1.0,2,3,1,0,1,2
2,2_Item Category,1,Equipment,3,Dept001,PR_10001003,2022-01-14,Equipment AB,V0002,3000.0,...,PR_10001001|PR_10001002|PR_10001003,2000.0,9000.0,1.0,2,3,1,0,1,2
3,2_Item Category,2,Equipment,2,Dept001,PR_10001004,2022-02-14,Equipment AB,V0003,6000.0,...,PR_10001004|PR_10001005,6000.0,14000.0,1.0,2,2,0,1,1,2
4,2_Item Category,2,Equipment,2,Dept001,PR_10001005,2022-02-15,Equipment AB,V0004,8000.0,...,PR_10001004|PR_10001005,6000.0,14000.0,1.0,2,2,0,1,1,2


## Combine Details

In [ ]:
df_sop_comb = pd.concat([df_sop_item, df_sop_itemcat])

In [ ]:
df_sop_comb

,Item Type,Grouping,Item,Count PR,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item Category,Vendor,Total Value,...,PR with similar items,Min_Value,Max_Value,Min_Date_Diff,Unique_Vendor,Count_Vendor,Flag_Threshold_Crossed,Flag_Same_Vendor,Flag_Min_Days,Score
0,1_Item,1,Equipment AA,2,Dept001,PR_10001001,2022-01-12,Equipment,V0001,2000.0,...,PR_10001001|PR_10001002,2000.0,6000.0,3.0,1,2,1,0,0,1
1,1_Item,1,Equipment AA,2,Dept001,PR_10001002,2022-01-15,Equipment,V0001,4000.0,...,PR_10001001|PR_10001002,2000.0,6000.0,3.0,1,2,1,0,0,1
2,1_Item,3,Equipment AB,2,Dept001,PR_10001004,2022-02-14,Equipment,V0003,6000.0,...,PR_10001004|PR_10001005,6000.0,14000.0,1.0,2,2,0,1,1,2
3,1_Item,3,Equipment AB,2,Dept001,PR_10001005,2022-02-15,Equipment,V0004,8000.0,...,PR_10001004|PR_10001005,6000.0,14000.0,1.0,2,2,0,1,1,2
0,2_Item Category,1,Equipment AA,3,Dept001,PR_10001001,2022-01-12,Equipment,V0001,2000.0,...,PR_10001001|PR_10001002|PR_10001003,2000.0,9000.0,1.0,2,3,1,0,1,2
1,2_Item Category,1,Equipment AA,3,Dept001,PR_10001002,2022-01-15,Equipment,V0001,4000.0,...,PR_10001001|PR_10001002|PR_10001003,2000.0,9000.0,1.0,2,3,1,0,1,2
2,2_Item Category,1,Equipment AB,3,Dept001,PR_10001003,2022-01-14,Equipment,V0002,3000.0,...,PR_10001001|PR_10001002|PR_10001003,2000.0,9000.0,1.0,2,3,1,0,1,2
3,2_Item Category,2,Equipment AB,2,Dept001,PR_10001004,2022-02-14,Equipment,V0003,6000.0,...,PR_10001004|PR_10001005,6000.0,14000.0,1.0,2,2,0,1,1,2
4,2_Item Category,2,Equipment AB,2,Dept001,PR_10001005,2022-02-15,Equipment,V0004,8000.0,...,PR_10001004|PR_10001005,6000.0,14000.0,1.0,2,2,0,1,1,2
5,2_Item Category,3,Equipment AA,2,Dept002,PR_10001006,2022-01-16,Equipment,V0001,4000.0,...,PR_10001006|PR_10001007,3000.0,7000.0,1.0,2,2,1,1,1,3


In [ ]:
# With the different methods, there will be Groupings with the exact same PR combination and items flagged, we can remove those
df_sop_comb.sort_values(['Grouping','Item'], inplace=True)

In [ ]:
df_sop_grouping_check = df_sop_comb\
.groupby(['Grouping','Item Type','PR with similar items','Score'])\
.agg({'Item': '|'.join}).reset_index()

In [ ]:
df_sop_grouping_check.columns = ['Grouping', 'Item Type', 'PR with similar items', 'Score', 'Items in Group']

In [ ]:
df_sop_grouping_check.sort_values(['PR with similar items','Items in Group','Item Type','Score'],
                                  ascending = [True, True, True, False],
                                  inplace=True)

In [ ]:
#Identify Groups with the same PRs and Items
df_sop_grouping_check[df_sop_grouping_check.duplicated(subset=['PR with similar items','Items in Group'],
                                                       keep=False)]

,Grouping,Item Type,PR with similar items,Score,Items in Group
3,3,1_Item,PR_10001004|PR_10001005,2,Equipment AB|Equipment AB
2,2,2_Item Category,PR_10001004|PR_10001005,2,Equipment AB|Equipment AB


In [ ]:
df_sop_grouping_unique = df_sop_grouping_check.drop_duplicates(subset=['PR with similar items','Items in Group'],
                                                       keep='first')
valid_groupings = set(df_sop_grouping_unique['Grouping'].tolist())

In [ ]:
df_sop_comb_final = df_sop_comb[df_sop_comb['Grouping'].isin(valid_groupings)]

In [ ]:
df_sop_comb_final

,Item Type,Grouping,Item,Count PR,Purchasing Department,Purchase Requisition Number,Purchase Requisition Date,Item Category,Vendor,Total Value,...,PR with similar items,Min_Value,Max_Value,Min_Date_Diff,Unique_Vendor,Count_Vendor,Flag_Threshold_Crossed,Flag_Same_Vendor,Flag_Min_Days,Score
0,1_Item,1,Equipment AA,2,Dept001,PR_10001001,2022-01-12,Equipment,V0001,2000.0,...,PR_10001001|PR_10001002,2000.0,6000.0,3.0,1,2,1,0,0,1
1,1_Item,1,Equipment AA,2,Dept001,PR_10001002,2022-01-15,Equipment,V0001,4000.0,...,PR_10001001|PR_10001002,2000.0,6000.0,3.0,1,2,1,0,0,1
0,2_Item Category,1,Equipment AA,3,Dept001,PR_10001001,2022-01-12,Equipment,V0001,2000.0,...,PR_10001001|PR_10001002|PR_10001003,2000.0,9000.0,1.0,2,3,1,0,1,2
1,2_Item Category,1,Equipment AA,3,Dept001,PR_10001002,2022-01-15,Equipment,V0001,4000.0,...,PR_10001001|PR_10001002|PR_10001003,2000.0,9000.0,1.0,2,3,1,0,1,2
2,2_Item Category,1,Equipment AB,3,Dept001,PR_10001003,2022-01-14,Equipment,V0002,3000.0,...,PR_10001001|PR_10001002|PR_10001003,2000.0,9000.0,1.0,2,3,1,0,1,2
5,2_Item Category,3,Equipment AA,2,Dept002,PR_10001006,2022-01-16,Equipment,V0001,4000.0,...,PR_10001006|PR_10001007,3000.0,7000.0,1.0,2,2,1,1,1,3
2,1_Item,3,Equipment AB,2,Dept001,PR_10001004,2022-02-14,Equipment,V0003,6000.0,...,PR_10001004|PR_10001005,6000.0,14000.0,1.0,2,2,0,1,1,2
3,1_Item,3,Equipment AB,2,Dept001,PR_10001005,2022-02-15,Equipment,V0004,8000.0,...,PR_10001004|PR_10001005,6000.0,14000.0,1.0,2,2,0,1,1,2
6,2_Item Category,3,Equipment AB,2,Dept002,PR_10001007,2022-01-17,Equipment,V0002,3000.0,...,PR_10001006|PR_10001007,3000.0,7000.0,1.0,2,2,1,1,1,3
